# Pre-processing

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data/spotify_songs.csv')

In [ ]:
display(data)
display(data.describe().T.style.background_gradient(cmap='YlGnBu'))

In [ ]:
# check for missing values
missing_values = data.isnull().sum()
display(missing_values)

# display the lines with missing values
missing_data = data[data.isnull().any(axis=1)]
display(missing_data)

In [ ]:
# Remove id columns
data = data.drop(columns=['track_id', 'track_album_id', "playlist_id"], axis=1)
# drop the missing values
data = data.dropna()
display(data)


In [ ]:
# transform track_album_release_date into datetime
data['track_album_release_date'] = pd.to_datetime(data['track_album_release_date'], format='mixed')

# transform categorical columns into categorical data type
categorical_cols = ['playlist_genre', 'playlist_subgenre', 'track_artist', 'playlist_name', 'track_album_name']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# transform the duration_ms into minutes
data['duration_s'] = data['duration_ms'] / 1000
data.drop(columns=['duration_ms'], inplace=True)

# For numeric columns that represent discrete values (like key and mode), convert to categorical
key_mapping = {
    0: 'C', 1: 'C♯/D♭', 2: 'D', 3: 'D♯/E♭', 4: 'E', 5: 'F',
    6: 'F♯/G♭', 7: 'G', 8: 'G♯/A♭', 9: 'A', 10: 'A♯/B♭', 11: 'B'
}
data['key'] = data['key'].map(key_mapping).astype('category')

data['mode'] = data['mode'].map({0: 'Minor', 1: 'Major'}).astype('category')



In [ ]:
# check for duplicates
duplicates = data.duplicated().sum()
display(duplicates)

In [ ]:
data.info()

# Descriptive Analysis

# 2. Réduction de dimension par ACP (Analyse en Composantes Principales)
Dans cette partie, nous allons effectuer une analyse en composantes principales (ACP) sur les données prétraitées. L'ACP est une technique de réduction de dimension qui permet de projeter les données d'origine dans un espace de dimension inférieure.
Nous avons gardé 20 variables et nous allons étudier s'il est possible de réduire la dimensionnalité de ces données tout en préservant un maximum d'information.

In [ ]:
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

### 2.1 Format des données
Ici on ne sélectionne que les variables quantitatives, en y ajoutant une variable qualitative (`playlist_genre`) pour voir si elle a un impact sur la projection des données. On garde au final 11 variables quantitatives et 1 variable qualitative.

On décide de normaliser les données pour que chaque variable ait une moyenne de 0 et un écart-type de 1. Cela est important car l'ACP est sensible à l'échelle des variables. On utilise la méthode `StandardScaler` de `sklearn` pour normaliser les données.

In [ ]:
# Select only the quantitative columns

qualisup = 'playlist_genre'

data_quanti = data.select_dtypes(include=['int64', 'float64'])
data_quanti[qualisup] = data[qualisup]
data_quanti = data_quanti.set_index(qualisup)

data_quanti.info()

In [ ]:
data_quanti[:5]

### 2.2 ACP

In [ ]:
from sklearn.decomposition import PCA
import prince 
from sklearn.preprocessing import StandardScaler

normalize_bool = True

if normalize_bool:
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_quanti)
else:
    data_scaled = data_quanti.values

pca = PCA(
    n_components=10,  # Number of components to keep
    random_state=1  # For reproducibility
)

projected = pca.fit_transform(data_scaled)

explained_variance = pca.explained_variance_ratio_

eig = pd.DataFrame(
    {
        "Dimension" : [f"PC{i+1}" for i in range(len(explained_variance))],
        "Variance" : np.round(pca.explained_variance_, 2),
        "% explained variance" : np.round(explained_variance*100, 1),
        "% cumulative variance" : np.round(np.cumsum(explained_variance)*100, 1)
    }
)
eig

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,2,1)
ax.bar(range(10), pca.explained_variance_ratio_[:10]*100, align='center',
        color='coral', ecolor='black')
ax.set_xticks(range(10))
ax.set_ylabel("Variance")
ax.set_title("", fontsize=35)
ax.set_title(u"Pourcentage de variance expliqué", fontsize=20)

ax = fig.add_subplot(1,2,2)
ax.plot(np.cumsum(pca.explained_variance_ratio_), color='coral', marker='o')
ax.hlines(0.80, 0, 10, colors='grey', linestyles='dashed', alpha=0.5)
ax.set_title(u'Pourcentage de variance expliqué cumulé', fontsize=20)

fig.suptitle(u"Résultat ACP", fontsize=25)
plt.show()

**Interprétation :** 

L’analyse de la variance expliquée montre que les **7 premières composantes principales** permettent de **représenter 80,1 % de la variance totale** du jeu de données. Cela signifie que l’essentiel de l’information contenue dans les **11 variables numériques initiales** (comme *danceability*, *energy*, *speechiness*, *tempo*, etc.) peut être résumé avec seulement 7 dimensions, ce qui représente une **réduction significative de la complexité** du dataset tout en conservant une bonne qualité descriptive.

Dans cette optique de réduction de dimension, il serait **pertinent de conserver ces 7 composantes principales** pour la suite des analyses (clustering, visualisation, classification), car elles capturent la structure principale des données tout en éliminant le "bruit".

Dans l’analyse factorielle, nous avons choisi d’**interpréter les trois premières composantes principales**, qui à elles seules expliquent 45 % de la variance totale. Elles offrent un bon compromis entre lisibilité et pertinence pour une visualisation ou une première analyse des relations entre les variables et les genres musicaux (*playlist_genre*).

In [ ]:
fig = plt.figure(figsize=(12,6))
box=plt.boxplot(projected[:,0:3],whis=100)
plt.title(u"Distribution des premières composantes", fontsize=20)
plt.show()

**Interprétation :**
- Le graphique ci-dessus montre la distribution des premières composantes principales. 

In [ ]:
pca_prince = prince.PCA(
    n_components=3,
    n_iter=3,
    rescale_with_mean=normalize_bool,
    rescale_with_std=normalize_bool,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=1
)
pca_prince = pca_prince.fit(
    data_quanti,
    sample_weight=None,
    column_weight=None,
    supplementary_columns=None
)

# Create a correlation matrix of the PCA compnenets
correlations = pca_prince.column_correlations
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='RdBu', center=0)
plt.title('Corrélations entre les variables et les composantes principales')
plt.show()


In [ ]:
# Récupération des coordonnées des variables (correlations avec les composantes principales)
coords = pca_prince.column_correlations

# Création de la figure et des sous-graphiques
fig, axes = plt.subplots(1, 3, figsize=(24, 8))

# Boucle pour créer les trois graphiques
for idx, (ax, (x_comp, y_comp)) in enumerate(zip(axes, [(0, 1), (1, 2), (0, 2)])):
    ax.grid(False)
    for i in range(coords.shape[0]):
        ax.arrow(0, 0, coords.iloc[i, x_comp], coords.iloc[i, y_comp], 
                 color='black', alpha=0.7, head_width=0.02, head_length=0.03)
        ax.text(coords.iloc[i, x_comp] * 1.1, coords.iloc[i, y_comp] * 1.1, 
                coords.index[i], color='black', ha='center', va='center')

    # Ajout du cercle de rayon 1
    ax.add_artist(plt.Circle((0, 0), radius=1, color='cornflowerblue', fill=False))

    # Ajout des axes
    ax.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    ax.axvline(0, color='gray', linestyle='--', linewidth=0.5)

    # Paramètres du graphique
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlabel(f'Composante principale {x_comp + 1}')
    ax.set_ylabel(f'Composante principale {y_comp + 1}')
    ax.set_title(f'Projection des variables: CP{x_comp + 1} vs CP{y_comp + 1}')

plt.tight_layout()
plt.show()

La table des correlations et les trois graphiques ci-dessus représentent les projections des features sur les trois premières composantes principales nous donne des informations sur la structure des données réduites.

- **Composante principale 1** : 
    - Les variables `energy (-0.91)`, `loudness (-0.80)` et `acousticness (+0.72)` sont linéairement corrélées avec la première composante principale, en soulignant que `energy` et `loudness` sont inversément corrélées avec `acousticness`. Cela indique que la CP1 oppose les morceaux **énergiques, forts en volume et peu acoustiques** (ex : rock, électro) aux morceaux **calmes, acoustiques et peu énergétiques** (ex : folk, classique).
- **Composante principale 2** : Sur le graphique de gauche, on remarque une opposition des variables `instrumentalness (+0.45)`, `duration_s (+0.38)` contre `danceability (-0.68)`, `valence (-0.62)`, `track_popularity (-0.37)`, `speechiness (-0.39)`. Cette composante principale oppose deux profils de morceaux :
    - D’un côté, les morceaux **instrumentaux, longs et peu populaires** (forte contribution de `instrumentalness` et `duration_s`), souvent associés à des genres comme le classique ou le jazz.
    - De l’autre, les chansons **courtes, dansantes, joyeuses et populaires** (forte contribution de `danceability`, `valence` et `track_popularity`), typiques de la pop ou de la musique de club. 
    - Enfin, cette opposition suggère que les morceaux avec des paroles marquées (`speechiness`) et une structure rythmique engageante (`danceability`) sont plus susceptibles de générer de la popularité.
- **Composantes principales 2 et 3** : Sur le graphique au centre, on peut extraire plusieurs informations :
    - Les morceaux dansants et joyeux (haute valence) s'opposent dans une moindre mesure aux morceaux de faible tempo. De plus, ces morceaux semblent avoir peu de versions live.
    - On observe aussi que les morceaux instrumentaux et longs ont généralement une faible popularité, tandis que les morceaux courts et peu instrumentaux sont souvent plus populaires, ce qui est typique de la musique pop, qui est souvent plus accessible et commerciale.
- **Composante principale 3** : La troisième composante (CP3) révèle un paradoxe : elle regroupe des morceaux à fort potentiel dansant (`danceability`) et mood positif (`valence`), mais qui restent peu populaires (`track_popularity`). Ces morceaux sont souvent instrumentaux (`instrumentalness`), longs (`duration_s`) et à tempo faible (`tempo`), ce qui les éloigne des standards des charts. Cette composante pourrait représenter des créations artistiques équilibrant danse et complexité, mais peinant à atteindre un large public.

Pour mieux comprendre à quoi correspond les type morceaux extraits par ces composantes principales, nous allons regarder les morceaux (individus) contribuant le plus à chacune des composantes principales.

In [ ]:
# display la mediane de notre dataset quantitatif à fins de comparaison
median_values = data_quanti.median()
median_values = median_values.to_frame(name='median')
median_values = median_values.reset_index()
median_values.columns = ['feature', 'median']
median_values['feature'] = median_values['feature'].astype('category')
median_values = median_values.set_index('feature')
median_values = median_values.sort_index()
median_values = median_values.T
median_values

#### Composante principale 1

Afin de mieux comprendre les profils musicaux mis en évidence par la première composante principale, nous allons examiner les morceaux qui y contribuent le plus fortement, positivement et négativement.

In [ ]:
# Calculate the contributions of individuals to the first principal component
contributions = projected[:, 0]

# Get the indices of the top 5 positive contributors
top_5_positive_indices = np.argsort(contributions)[-5:]

# Get the indices of the top 5 negative contributors
top_5_negative_indices = np.argsort(contributions)[:5]

# Extract the corresponding rows from the original dataset for positive contributions
top_5_positive_tracks = data_quanti.iloc[top_5_positive_indices].copy()
top_5_positive_tracks['track_name'] = data.iloc[top_5_positive_indices]['track_name'].values
top_5_positive_tracks['track_artist'] = data.iloc[top_5_positive_indices]['track_artist'].values
top_5_positive_tracks['contribution'] = contributions[top_5_positive_indices]
top_5_positive_tracks['playlist_subgenre'] = data.iloc[top_5_positive_indices]['playlist_subgenre'].values
# sort by contribution
top_5_positive_tracks = top_5_positive_tracks.sort_values(by='contribution', ascending=False)

# Extract the corresponding rows from the original dataset for negative contributions
top_5_negative_tracks = data_quanti.iloc[top_5_negative_indices].copy()
top_5_negative_tracks['track_name'] = data.iloc[top_5_negative_indices]['track_name'].values
top_5_negative_tracks['track_artist'] = data.iloc[top_5_negative_indices]['track_artist'].values
top_5_negative_tracks['contribution'] = contributions[top_5_negative_indices]
top_5_negative_tracks['playlist_subgenre'] = data.iloc[top_5_negative_indices]['playlist_subgenre'].values

# Display the top 5 positive and negative tracks with additional information
print("Top 5 Positive Contributions (PC1):")
display(top_5_positive_tracks)

print("Top 5 Negative Contributions (PC1) :")
display(top_5_negative_tracks)

Pour mieux cerner les types de morceaux représentés aux extrémités de la **première composante principale (PC1)**, nous avons identifié les individus (chansons) ayant les **contributions les plus élevées**, positives comme négatives.

- **Du côté des contributions positives**, on retrouve majoritairement des morceaux **rock, hard rock ou pop rock** très énergiques et puissants tels que *American Idiot* (Green Day), *Beauty Queen* (BLVK SWVN) ou *ATTENTION ATTENTION* (Shinedown). Ces morceaux sont caractérisés par une **énergie élevée**, une **forte intensité sonore (loudness)** et une **faible acoustique**, ce qui confirme bien la structure mise en évidence par la CP1. Notons aussi *This Is How We Do It* (Montell Jordan), un morceau R&B énergique, qui se distingue des autres par son genre mais partage les mêmes caractéristiques acoustiques.

- **À l’opposé**, les morceaux à contribution très négative sur PC1 sont des titres à **forte acoustique**, **peu énergiques** et **très faibles en loudness**. Il s'agit notamment de sons **ambiants, relaxants ou naturels**, comme *Peaceful Forest* ou *Tropical Rainforest at Dawn*, mais aussi de titres R&B ou indie très doux (*Small* de chloe moriondo). Ces morceaux incarnent l'autre extrémité de la CP1 : **des chansons calmes, acoustiques et à faible énergie**, souvent issues de sous-genres comme *tropical*, *indie poptimism* ou *new jack swing*.

Cette opposition renforce l’interprétation de la **CP1 comme un axe énergie / intensité sonore vs. calme / acoustique**, pertinent pour distinguer deux grandes familles de styles musicaux dans le dataset.

#### Composante principale 2

Pour approfondir l’interprétation de la **deuxième composante principale**, nous allons analyser les morceaux qui y contribuent le plus fortement — positivement comme négativement.

In [ ]:
# Calculate the contributions of individuals to the second principal component
contributions_pc2 = projected[:, 1]

# Get the indices of the top 5 positive contributors for PC2
top_5_positive_indices_pc2 = np.argsort(contributions_pc2)[-5:]

# Get the indices of the top 5 negative contributors for PC2
top_5_negative_indices_pc2 = np.argsort(contributions_pc2)[:5]

# Extract the corresponding rows from the original dataset for positive contributions (PC2)
top_5_positive_tracks_pc2 = data_quanti.iloc[top_5_positive_indices_pc2].copy()
top_5_positive_tracks_pc2['track_name'] = data.iloc[top_5_positive_indices_pc2]['track_name'].values
top_5_positive_tracks_pc2['track_artist'] = data.iloc[top_5_positive_indices_pc2]['track_artist'].values
top_5_positive_tracks_pc2['contribution'] = contributions_pc2[top_5_positive_indices_pc2]
top_5_positive_tracks_pc2['playlist_subgenre'] = data.iloc[top_5_positive_indices_pc2]['playlist_subgenre'].values
# Sort by contribution
top_5_positive_tracks_pc2 = top_5_positive_tracks_pc2.sort_values(by='contribution', ascending=False)

# Extract the corresponding rows from the original dataset for negative contributions (PC2)
top_5_negative_tracks_pc2 = data_quanti.iloc[top_5_negative_indices_pc2].copy()
top_5_negative_tracks_pc2['track_name'] = data.iloc[top_5_negative_indices_pc2]['track_name'].values
top_5_negative_tracks_pc2['track_artist'] = data.iloc[top_5_negative_indices_pc2]['track_artist'].values
top_5_negative_tracks_pc2['contribution'] = contributions_pc2[top_5_negative_indices_pc2]
top_5_negative_tracks_pc2['playlist_subgenre'] = data.iloc[top_5_negative_indices_pc2]['playlist_subgenre'].values

# Display the top 5 positive and negative tracks with additional information for PC2
print("Top 5 Positive Contributions (PC2):")
display(top_5_positive_tracks_pc2)

print("Top 5 Negative Contributions (PC2):")
display(top_5_negative_tracks_pc2)

**Côté contributions positives**, on retrouve des titres principalement **rap et latino**, tels que *Suge* de DaBaby ou *LAX* de B0nds. Ces morceaux sont :
- **courts**,
- **dansants** (haute `danceability`),
- avec une **valence élevée** (émotion positive),
- mais également avec un certain niveau de **speechiness**, notamment pour les titres rap.

Ces morceaux partagent donc des caractéristiques propres aux chansons **énergétiques, rythmées et populaires**, souvent taillées pour le streaming, avec des formats courts, accrocheurs et directs.

**À l’opposé**, les morceaux ayant une **forte contribution négative à PC2** sont très différents : on retrouve des **paysages sonores naturels, ambiants ou instrumentaux** comme *Rain Forest and Tropical Beach Sound*, *Caribbean Thunderstorm*, ou encore *Battlement*. Ces titres sont :
- **longs**,
- **instrumentaux** (forte `instrumentalness`),
- avec une **faible valence** et **peu de parole**,
- et souvent issus de sous-genres comme *tropical*, *album rock*, ou *ambient*.

Cela confirme l’interprétation initiale de la PC2 comme un **axe opposant la musique instrumentale, longue et contemplative** à une musique **populaire, dansante et rythmée**.


#### Composante principale 3

In [ ]:
# Calculate the contributions of individuals to the third principal component
contributions_pc3 = projected[:, 2]

# Get the indices of the top 5 positive contributors for PC3
top_5_positive_indices_pc3 = np.argsort(contributions_pc3)[-5:]

# Get the indices of the top 5 negative contributors for PC3
top_5_negative_indices_pc3 = np.argsort(contributions_pc3)[:5]

# Extract the corresponding rows from the original dataset for positive contributions (PC3)
top_5_positive_tracks_pc3 = data_quanti.iloc[top_5_positive_indices_pc3].copy()
top_5_positive_tracks_pc3['track_name'] = data.iloc[top_5_positive_indices_pc3]['track_name'].values
top_5_positive_tracks_pc3['track_artist'] = data.iloc[top_5_positive_indices_pc3]['track_artist'].values
top_5_positive_tracks_pc3['contribution'] = contributions_pc3[top_5_positive_indices_pc3]
top_5_positive_tracks_pc3['playlist_subgenre'] = data.iloc[top_5_positive_indices_pc3]['playlist_subgenre'].values
# Sort by contribution
top_5_positive_tracks_pc3 = top_5_positive_tracks_pc3.sort_values(by='contribution', ascending=False)

# Extract the corresponding rows from the original dataset for negative contributions (PC3)
top_5_negative_tracks_pc3 = data_quanti.iloc[top_5_negative_indices_pc3].copy()
top_5_negative_tracks_pc3['track_name'] = data.iloc[top_5_negative_indices_pc3]['track_name'].values
top_5_negative_tracks_pc3['track_artist'] = data.iloc[top_5_negative_indices_pc3]['track_artist'].values
top_5_negative_tracks_pc3['contribution'] = contributions_pc3[top_5_negative_indices_pc3]
top_5_negative_tracks_pc3['playlist_subgenre'] = data.iloc[top_5_negative_indices_pc3]['playlist_subgenre'].values

# Display the top 5 positive and negative tracks with additional information for PC3
print("Top 5 Positive Contributions (PC3):")
display(top_5_positive_tracks_pc3)

print("Top 5 Negative Contributions (PC3):")
display(top_5_negative_tracks_pc3)

La **troisième composante principale** met en lumière une tension plus subtile entre deux types de morceaux aux caractéristiques inattendues.

**Du côté des contributions positives**, on retrouve des titres de **pop et R&B calmes et acoustiques** comme *raindrops (an angel cried)* (Ariana Grande) ou *You Are The Reason* (Calum Scott). Ces chansons ont :
- une **acousticness élevée** (acapela, guitare acoustique, piano),
- un **tempo légèrement plus rapide que la médiane des morceaux**,
- une **faible énergie**, mais un **potentiel émotionnel fort** (`valence` variable).

Comme suggéré par la PC3, ces morceaux rencontrent un certain succès, illustrant un profil de chansons émotionnelles, accessibles et bien produites, souvent interprétées par des artistes grand public au style sobre et expressif.

**Les contributions négatives**, quant à elles, sont largement dominées par des morceaux **EDM ou latino instrumentaux**, comme *I Feel Love* ou *Chase*. Ces morceaux sont :
- **longs**,
- très **instrumentaux**,
- **énergiques** mais souvent **moins "accessibles" émotionnellement** (valence très élevée mais peu de paroles, structure répétitive).

Ils présentent également une popularité extrêmement faible, allant de 0 à 8. Ces productions s’adressent probablement à un public averti ou sont conçues pour des usages spécifiques (DJ sets, ambiances electro), ce qui les éloigne des standards de la musique grand public.

Ces observations confirment que la **PC3 oppose des créations acoustiques à forte charge émotionnelle** à des morceaux **instrumentaux, électroniques, longs**, aux dynamiques parfois complexes ou répétitives.

On remarque que le signe des contribution est inversé par rapport à la PCA réalisé en R uniquement pour cette dimension.

## Multiple Factorial Analysis (MCA)

**1. Variables catégorielles à utiliser dans la MCA**
Voici les variables qualitatives que l'on pourrait envisager d'utiliser pour la MCA sur le dataset Spotify :  
- `track_artist`  
- `track_album_name` (attention à trop de modalités rares, peut-être garder que les plus fréquentes ou ne pas inclure à cause du grand nombre d’artistes)  
- `playlist_name`  
- `playlist_genre`  
- `playlist_subgenre`  
- `key`  
- `mode`  

---

**2. Questions que la MCA peut aider à explorer**

**a) Quels sont les groupes/cluster de modalités similaires ?**
- Est-ce que certains genres et sous-genres de playlists s’associent fréquemment ?  
- Certains "modes" (majeur/minor) sont-ils plus fréquents dans certains genres ?  
- Y a-t-il des clés (`key`) musicales qui sont typiques de certains genres ou playlists ?  

La MCA permettra de représenter graphiquement (biplot) ces modalités et d’identifier des associations fortes.

**b) Est-ce que certains artistes ou playlists ont un profil qualitatif particulier ?**
- Par exemple, certains artistes seraient-ils associés à un genre et sous-genre spécifiques, ou à un mode particulier ?  
- Y a-t-il des clusters d’artistes / playlists qui partagent des caractéristiques particulières (clé, mode, genre) ?

**3. Comment interpréter la MCA ici**

- **Axes factoriels** : Chaque axe correspond à une dimension qui résume des associations fortes entre modalités. Par exemple, un axe peut opposer les genres "Rock" à "Pop", ou des clés majeures à mineures.
- **Modalités proches dans l’espace** : Modalités proches signifient qu’elles co-apparaissent souvent dans les observations (ex. certains genres + mode majeur).
- **Observation** : Si tu represents les observations (chansons) dans l’espace MCA, celles proches partagent des profils catégoriels similaires.

---

**4. Utilisations concrètes/academic use cases**

- **Profil des genres musicaux** : Comprendre quels modes/clés/sous-genres caractérisent les genres populaires sur Spotify.  
- **Segmentation qualitative des playlists** : Y a-t-il des types de playlists/musiques qui partagent des caractéristiques qualitatives communes ?  
- **Analyse de diversité** : Mesurer dans quelle mesure certains artistes/genres sont hétérogènes ou homogènes quant à leurs caractéristiques catégorielles.  
- **Préparation à une classification** : Par exemple, combiner le résultat de la PCA (variables numériques) avec la MCA (variables qualitatives) dans une analyse factorielle mixte ou pour enrichir un modèle prédictif.

---

**5. Exemple de questions précises à poser**

- Les genres musicaux sont-ils liés à certaines tonalités ou modes ?  
- Les sous-genres présents dans la même playlist sont-ils proches ou éloignés dans l’espace MCA ?  
- Y a-t-il des clés rares ou des modes minoritaires associés à certains genres uniquement ?  
- Peut-on détecter des groupes de playlists ou artistes avec des profils qualitatifs similaires ?  

---

**En conclusion**
La MCA t’aide surtout à **explorer et visualiser les relations entre variables qualitatives** et leurs modalités sur ton dataset Spotify, ce qui complète bien la PCA sur les variables numériques. C’est une étape utile pour comprendre la structure qualitative de tes données avant d’envisager une modélisation supervisée ou une analyse plus approfondie.

---

Si tu veux, je peux aussi te fournir un exemple de code Python pour réaliser une MCA avec `prince` ou en R avec `FactoMineR` sur ton dataset.